In [2]:
import os
import json
import pandas as pd
import emoji

# Specify the directory containing the JSON files
json_directory = 'C:\\Users\\djsma\\Downloads\\inbox'

def process_json_file(file_path, folder_name):
    # Read the JSON file
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    # Extract the messages from the data
    messages = data['messages']

    # Create a DataFrame from the messages
    df = pd.DataFrame(messages)

    # Add a unique identifier column based on the index
    df.insert(0, 'message_id', range(1, len(df) + 1))

    # Add the folder name as a new column
    df['folder_name'] = folder_name

    # Function to decode messages
    def decode_message(content):
        if pd.isna(content):
            return ""
        try:
            content = content.encode('latin1').decode('unicode-escape')
        except UnicodeDecodeError:
            content = content.encode('utf-8').decode('utf-8')
        return emoji.demojize(content)

    # Apply the decode_message function to the 'content' field
    df['message'] = df.apply(
        lambda x: decode_message(x['content']) if 'content' in x and isinstance(x['content'], str) else "", axis=1
    )

    return df

# Initialize an empty list to store the DataFrames
df_list = []

# Iterate over all subdirectories in the main directory
for folder_name in os.listdir(json_directory):
    account_dir = os.path.join(json_directory, folder_name)
    if os.path.isdir(account_dir):
        json_file = os.path.join(account_dir, 'message_1.json')
        if os.path.isfile(json_file):
            df = process_json_file(json_file, folder_name)
            df_list.append(df)

# Concatenate all DataFrames into a single DataFrame
final_df = pd.concat(df_list, ignore_index=True)

# Select specific columns and rename them
final_df = final_df[['message_id', 'folder_name', 'sender_name', 'timestamp_ms', 'message']]
final_df.rename(columns={'timestamp_ms': 'timestamp'}, inplace=True)

# Save the DataFrame to a CSV file
output_file = 'instagram_messages1.csv'
final_df.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"DataFrame saved to {output_file}")


C:\Users\djsma\AppData\Local\Temp\ipykernel_46644\750875316.py:31: DeprecationWarning: invalid escape sequence '\_'
  content = content.encode('latin1').decode('unicode-escape')
C:\Users\djsma\AppData\Local\Temp\ipykernel_46644\750875316.py:31: DeprecationWarning: invalid escape sequence '\â'
  content = content.encode('latin1').decode('unicode-escape')


DataFrame saved to instagram_messages1.csv
